In [1]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

Using TensorFlow backend.


In [2]:
from utils import vectorize_sequences

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

import numpy as np

y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [3]:
from keras import models
from keras import layers
import keras

def create_model(num_layers=2, num_dimensions=16, activation='relu', optimizer='rmsprop', loss_func='binary_crossentropy'):
    model = models.Sequential()
    model.add(layers.Dense(num_dimensions, activation=activation, input_shape=(10000,)))
    for i in range(num_layers - 1):
        model.add(layers.Dense(num_dimensions, activation=activation))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer=optimizer,
                  loss=loss_func,
                  metrics=['accuracy'])
    return model

def get_history(model, epochs=20):
    class WeightsHistory(keras.callbacks.Callback):
        def on_train_begin(self, logs):
            self.weights = []

        def on_epoch_end(self, batch, logs):
            self.weights.append(self.model.get_weights())
    
    weights_history = WeightsHistory()

    history = model.fit(
        x_train,
        y_train,
        epochs=epochs,
        batch_size=512,
        validation_data=(x_test, y_test),
        callbacks=[weights_history]
    )
    
    return (history, weights_history)

def get_best_epoch(history):
    minimum_loss = min(history.history['val_loss'])
    return history.history['val_loss'].index(minimum_loss)

def evaluate_model(model, epochs):
    model.fit(x_train, y_train, epochs=epochs, batch_size=512)
    return model.evaluate(x_test, y_test)


In [4]:
# model = create_model(num_layers=1, num_dimensions=8, activation='tanh')
model = create_model()
history, weights_history = get_history(model, 10)
best_epoch = get_best_epoch(history)
best_weights = weights_history.weights[best_epoch]
model.set_weights(best_weights)
model.evaluate(x_test, y_test)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 4s 146us/step - loss: 0.4496 - acc: 0.8164 - val_loss: 0.3340 - val_acc: 0.8793
Epoch 2/10
25000/25000 [==============================] - 3s 113us/step - loss: 0.2552 - acc: 0.9094 - val_loss: 0.3169 - val_acc: 0.8709
Epoch 3/10
25000/25000 [==============================] - 2s 92us/step - loss: 0.1969 - acc: 0.9299 - val_loss: 0.2821 - val_acc: 0.8882
Epoch 4/10
25000/25000 [==============================] - 2s 89us/step - loss: 0.1669 - acc: 0.9408 - val_loss: 0.2946 - val_acc: 0.8832
Epoch 5/10
25000/25000 [==============================] - 2s 93us/step - loss: 0.1431 - acc: 0.9485 - val_loss: 0.3221 - val_acc: 0.8782
Epoch 6/10
25000/25000 [==============================] - 2s 95us/step - loss: 0.1266 - acc: 0.9560 - val_loss: 0.3491 - val_acc: 0.8725
Epoch 

[0.2821160471057892, 0.8882]